# A2 · Archived Ensemble (Исторический пайплайн)

**Что это.** Архивный ансамбль: **RuBERT (coarse)** → **OOD по CLS** → **Cross-Encoder rerank** → **Sub-классы на BERT-головах** → эвристики (fuzzy-hints).

**Почему в архиве**

- Высокая латентность и дорогая поддержка (несколько тяжёлых голов).
    
- Заменён на **тонкие головы** (логрег/GBM) по замороженным эмбеддингам.
    

**Зачем оставляем ноутбук**

- Точка сравнения (абляция) и историческая репликация.
    
- Песочница для R&D (при необходимости вернуть CE).
    

---

## Наблюдаемые показатели из ноутбука

_На sanity-наборах и стресс-тестах, которые запускались внутри ноутбука:_

- Sanity (~23 примера):
    
    - **Other rate ≈ 43.5%**, **Sub-coverage ≈ 21.7%**, **OOD ≈ 0%**.
        
    - Покрытие саб-меток внутри «Квартир»/«Авто» — 100% на попавших в эти coarse-классы примерах.
        
- Стресс-тест (защумлённые строки): **Other rate ≈ 32.5%**.
    
- A/B-гипотеза (лексикон для авто/телефонов):
    
    - **Other: 55.8% → 23.3%**
        
    - **Авто: 2.3% → 34.9%**
        
    - Изменено предсказаний: **14**; для **13/14** промоутнутых в «Авто» присвоилась марка.
        

> Полноценные сводные метрики по корпусу (Macro-F1 и пр.) в этом ноутбуке не посчитались, потому что «боевой» csv не был найден в окружении. Для честного сравнения с финальным пайплайном лучше прогнать общий `eval.csv` через свежий `05_Inference_Pipeline_Final.ipynb`.


## 1. Импорт, устройство, пути

In [3]:

import os, json, warnings, re
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from IPython.display import display

warnings.filterwarnings("ignore")

DEVICE = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print("DEVICE:", DEVICE)

ARTIFACTS_ROOT = Path("../../data")

MAIN_MODEL_DIR = ARTIFACTS_ROOT / "rubert_cls_model"
CE_MODEL_DIR = ARTIFACTS_ROOT / "cross_encoder_rubert"
SUB_AUTOS_DIR = ARTIFACTS_ROOT / "subclf_rubert_autos"
SUB_APART_DIR = ARTIFACTS_ROOT / "subclf_rubert_apart"
THRESH_PATH = ARTIFACTS_ROOT / "inference_thresholds.json"

assert MAIN_MODEL_DIR.exists(), f"Нет папки модели: {MAIN_MODEL_DIR}"
assert THRESH_PATH.exists(),    f"Нет файла порогов: {THRESH_PATH}"

T = json.load(open(THRESH_PATH, "r", encoding="utf-8"))

TAU_OTHER = float(T.get("main_tau_other", 0.35))
TAU_HIGH = float(T.get("main_tau_high", 0.75))

OOD = T.get("ood", {})
ALPHA_FUSE = float(T.get("alpha", T.get("ood", {}).get("alpha", 1.0)))
z_thr = float(OOD.get("z_thr", 8.0))
mu = float(OOD.get("mu", 0.0))
sigma = float(OOD.get("sigma", 1.0))
thr_raw = OOD.get("threshold", None)
try: thr_raw = float(thr_raw) if thr_raw is not None else None
except: thr_raw = None

SUB = T.get("sub", {})
TAU_AUTOS  = float(SUB.get("autos_tau", 0.8))
TAU_APART  = float(SUB.get("apart_tau", 0.45))

# Пер-классные пороги (если в JSON)
TAU_OTHER_BY_LABEL = T.get("tau_other_by_label", {})  # { "Легковые автомобили": 0.30, ... }

print("tau_other:", TAU_OTHER, "| tau_high:", TAU_HIGH, "| alpha:", ALPHA_FUSE)
print("OOD -> z_thr:", z_thr, "| mu:", mu, "| sigma:", sigma, "| threshold:", thr_raw)
print("Sub -> autos_tau:", TAU_AUTOS, "| apart_tau:", TAU_APART)


DEVICE: cpu
tau_other: 0.2825582677025756 | tau_high: 0.75 | alpha: 1.0
OOD -> z_thr: 7.947675119608771 | mu: 863.1790571325726 | sigma: 675.8236249816604 | threshold: 4141.648698247054
Sub -> autos_tau: 0.6 | apart_tau: 0.3


## 2. Загрузка моделей

In [2]:

def normalize_idmaps(cfg):
    id2label = getattr(cfg, "id2label", None) or {}
    label2id = getattr(cfg, "label2id", None) or {}
    out = {}
    for k, v in id2label.items():
        try: out[int(k)] = str(v)
        except: pass
    if not out and label2id:
        for lbl, idx in label2id.items():
            try: out[int(idx)] = str(lbl)
            except: pass
    if not out:
        n = getattr(cfg, "num_labels", 0)
        out = {i: f"LABEL_{i}" for i in range(n)}
    mx = max(out.keys())
    for i in range(mx + 1):
        out.setdefault(i, f"LABEL_{i}")
    return out, {v:i for i,v in out.items()}

def _to_device(m): return m.to(DEVICE).eval()

def load_main_model(path):
    tok = AutoTokenizer.from_pretrained(path)
    mdl = AutoModelForSequenceClassification.from_pretrained(path)
    id2label, label2id = normalize_idmaps(mdl.config)
    base = AutoModel.from_pretrained(path)
    return tok, _to_device(mdl), _to_device(base), id2label, label2id

def load_ce_model(path):
    if not Path(path).exists():
        print("CE отсутствует — шаги CE будут пропущены.")
        return None, None
    tok = AutoTokenizer.from_pretrained(path)
    mdl = AutoModelForSequenceClassification.from_pretrained(path)
    return tok, _to_device(mdl)

def load_sub_model(path):
    if not Path(path).exists():
        return None, None, {}
    tok = AutoTokenizer.from_pretrained(path)
    mdl = AutoModelForSequenceClassification.from_pretrained(path)
    id2label, _ = normalize_idmaps(mdl.config)
    return tok, _to_device(mdl), id2label

main_tok, main_mdl, main_base, MAIN_ID2LABEL, MAIN_LABEL2ID = load_main_model(MAIN_MODEL_DIR)
ce_tok, ce_mdl = load_ce_model(CE_MODEL_DIR)
sub_auto_tok, sub_auto_mdl, SUB_AUTO_ID2LABEL = load_sub_model(SUB_AUTOS_DIR)
sub_aprt_tok, sub_aprt_mdl, SUB_APRT_ID2LABEL = load_sub_model(SUB_APART_DIR)

LABELS = [MAIN_ID2LABEL[i] for i in sorted(MAIN_ID2LABEL.keys())] if MAIN_ID2LABEL else None

def get_tau_other(lbl: str) -> float:
    if lbl in TAU_OTHER_BY_LABEL:
        try: return float(TAU_OTHER_BY_LABEL[lbl])
        except: pass
    if ce_mdl is None:
        l = (lbl or "").lower()
        if any(k in l for k in ["легков", "авто", "смартф"]):
            return min(0.30, TAU_OTHER)  # смягчаем при отсутствии CE
    return TAU_OTHER

print("Главных классов:", len(LABELS) if LABELS else "—")


Главных классов: 50


## 3. Вспомогательные функции

In [3]:

from rapidfuzz import process, fuzz

def normalize_text(s): return str(s).strip()

def cls_embed_raw_and_unit(texts, tokenizer, base_model, batch_size=16, max_length=256):
    raw_list, unit_list = [], []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = base_model(**enc, output_hidden_states=True)
            cls = out.last_hidden_state[:,0,:]            # RAW CLS
            unit = torch.nn.functional.normalize(cls, p=2, dim=-1)  # UNIT CLS
        raw_list.append(cls.detach().cpu())
        unit_list.append(unit.detach().cpu())
    raw = torch.cat(raw_list, dim=0).numpy()
    unit = torch.cat(unit_list, dim=0).numpy()
    return raw, unit

def main_infer(texts, top_k=5, batch_size=16, max_length=256):
    probs_all, logits_all = [], []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = main_tok(batch, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            o = main_mdl(**enc).logits
            p = torch.softmax(o, dim=-1)
        probs_all.append(p.detach().cpu().numpy())
        logits_all.append(o.detach().cpu().numpy())
    probs = np.vstack(probs_all)
    logits = np.vstack(logits_all)
    topk_idx = np.argsort(-probs, axis=1)[:, :top_k]
    topk_lbl = [[MAIN_ID2LABEL.get(int(i), f"LABEL_{int(i)}") for i in row] if MAIN_ID2LABEL else None for row in topk_idx]
    topk_pr  = np.take_along_axis(probs, topk_idx, axis=1)
    return probs, logits, topk_idx, topk_lbl, topk_pr

def ood_score_from_raw(raw_vecs):
    d2 = (raw_vecs ** 2).sum(axis=1)    # ||raw||^2
    z = (d2 - mu) / (sigma if sigma != 0 else 1.0)
    return d2, z

def ce_pair_scores(texts, labels):
    if ce_mdl is None: return None
    left, right = [], []
    for t, L in zip(texts, labels):
        for l in (L or []):
            left.append(t); right.append(l)
    if not left: return [None]*len(texts)
    enc = ce_tok(left, right, padding=True, truncation=True, max_length=256, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = ce_mdl(**enc).logits
        if logits.shape[-1] == 1:
            sc = torch.sigmoid(logits.squeeze(-1)).detach().cpu().numpy()
        elif logits.shape[-1] == 2:
            sc = torch.softmax(logits, dim=-1)[:,1].detach().cpu().numpy()
        else:
            sc = torch.softmax(logits, dim=-1).max(dim=-1).values.detach().cpu().numpy()
    res, idx = [], 0
    for L in labels:
        n = len(L or [])
        res.append(sc[idx:idx+n] if n else None)
        idx += n
    return res

def fuse_with_ce(main_probs, topk_idx, ce_scores, alpha=1.0):
    if ce_scores is None: return main_probs
    fused = main_probs.copy(); eps = 1e-9
    for r, (idxs, ce_sc) in enumerate(zip(topk_idx, ce_scores)):
        if ce_sc is None: continue
        boost = np.exp(alpha * np.clip(np.asarray(ce_sc), 0.0, 1.0))
        fused[r, idxs] *= boost
        s = fused[r].sum()
        if s > eps: fused[r] /= s
    return fused


## 4. Саб-классы: строгий роутинг

In [4]:

ALLOWED_AUTOS = {"Легковые автомобили"}
ALLOWED_APARTS = {"Квартиры — аренда", "Квартиры — продажа"}

def is_autos_allowed(lbl):  return lbl in ALLOWED_AUTOS
def is_aparts_allowed(lbl): return lbl in ALLOWED_APARTS

def apart_raw_to_tags(raw: str):
    if not raw: return []
    s = str(raw).lower()
    tags = []
    if s.startswith("продажа_"):
        n = s.split("_", 1)[1]
        tags = ["квартира: продажа", f"комнат: {n}"]
    elif s.startswith("аренда_"):
        n = s.split("_", 1)[1]
        tags = ["квартира: аренда", f"комнат: {n}"]
    elif s.startswith("студия_"):
        mode = s.split("_", 1)[1]
        mode = "продажа" if "продаж" in mode else "аренда"
        tags = ["квартира-студия", mode]
    else:
        parts = s.split("_")
        for p in parts:
            if p.isdigit(): tags.append(f"комнат: {p}")
            else: tags.append(p)
    return tags

def tags_to_human_apart(tags):
    if not tags: return None
    rooms = next((t for t in tags if t.startswith("комнат:")), None)
    mode = "продажа" if any("продажа" in t for t in tags) else ("аренда" if any("аренда" in t for t in tags) else None)
    studio = any("студия" in t for t in tags)
    parts = []
    parts.append("Квартира-студия" if studio else "Квартира")
    if mode: parts.append(f" — {mode}")
    if rooms:
        n = rooms.split(":")[1].strip()
        if n.isdigit(): parts.append(f", {n}-комнатная")
    return "".join(parts)

def tags_to_human_auto(brand):
    if not brand: return None
    return f"Авто — {brand}"

def sub_infer(texts, coarse_labels):
    autos_out, apart_out = [None]*len(texts), [None]*len(texts)

    if sub_auto_mdl is not None:
        idxs = [i for i,l in enumerate(coarse_labels) if is_autos_allowed(l)]
        if idxs:
            batch = [texts[i] for i in idxs]
            enc = sub_auto_tok(batch, padding=True, truncation=True, max_length=128, return_tensors="pt").to(DEVICE)
            with torch.no_grad():
                p = torch.softmax(sub_auto_mdl(**enc).logits, dim=-1).detach().cpu().numpy()
            labels = [SUB_AUTO_ID2LABEL.get(int(i), f"AUTOS_{int(i)}") for i in np.argmax(p, axis=1)]
            scores = np.max(p, axis=1)
            for k, i in enumerate(idxs):
                autos_out[i] = (labels[k], float(scores[k]))

    if sub_aprt_mdl is not None:
        idxs = [i for i,l in enumerate(coarse_labels) if is_aparts_allowed(l)]
        if idxs:
            batch = [texts[i] for i in idxs]
            enc = sub_aprt_tok(batch, padding=True, truncation=True, max_length=128, return_tensors="pt").to(DEVICE)
            with torch.no_grad():
                p = torch.softmax(sub_aprt_mdl(**enc).logits, dim=-1).detach().cpu().numpy()
            labels = [SUB_APRT_ID2LABEL.get(int(i), f"APART_{int(i)}") for i in np.argmax(p, axis=1)]
            scores = np.max(p, axis=1)
            for k, i in enumerate(idxs):
                apart_out[i] = (labels[k], float(scores[k]))

    return autos_out, apart_out


## 5. Предсказание

In [5]:

def predict_texts(texts, top_k=5, tau_high=TAU_HIGH, alpha=ALPHA_FUSE):
    texts = [normalize_text(t) for t in texts]
    probs, _, topk_idx, topk_lbl, _ = main_infer(texts, top_k=top_k)

    raw, unit = cls_embed_raw_and_unit(texts, main_tok, main_base)
    dist2, z = ood_score_from_raw(raw)
    use_thr = isinstance(thr_raw, (int, float))
    ood_by_thr = (dist2 > thr_raw) if use_thr else np.zeros_like(dist2, dtype=bool)
    is_ood = ood_by_thr | (z > z_thr)

    max_p = probs.max(axis=1)
    need_ce = (max_p < tau_high) & (~is_ood)
    if ce_mdl is not None and need_ce.any():
        ce_input_labels = [lbls if flag else [] for lbls, flag in zip(topk_lbl, need_ce)]
        ce_scores_all = ce_pair_scores(texts, ce_input_labels)
        ce_scores = [sc if flag else None for flag, sc in zip(need_ce, ce_scores_all)]
        probs = fuse_with_ce(probs, topk_idx, ce_scores, alpha=alpha)
        max_p = probs.max(axis=1)

    pred_idx = probs.argmax(axis=1)
    pred_lbl = [MAIN_ID2LABEL.get(int(i), f"LABEL_{int(i)}") for i in pred_idx]

    final_lbl, final_score = [], []
    for lbl, p, ood_flag in zip(pred_lbl, max_p, is_ood):
        tau = get_tau_other(lbl)
        if ood_flag or p < tau:
            final_lbl.append("Other"); final_score.append(float(p))
        else:
            final_lbl.append(lbl);     final_score.append(float(p))

    autos_sub, apart_sub = sub_infer(texts, final_lbl)
    sub_raw, sub_score, sub_tags, sub_label_hr = [], [], [], []
    for lbl, auto, apart in zip(final_lbl, autos_sub, apart_sub):
        if lbl == "Other":
            sub_raw.append(None); sub_score.append(None); sub_tags.append(None); sub_label_hr.append(None)
        elif auto is not None and auto[1] >= TAU_AUTOS:
            brand = auto[0]
            sub_raw.append(brand); sub_score.append(float(auto[1]))
            tags = [f"марка: {brand}"]; sub_tags.append(tags); sub_label_hr.append(tags_to_human_auto(brand))
        elif apart is not None and apart[1] >= TAU_APART:
            raw = apart[0]; tags = apart_raw_to_tags(raw)
            sub_raw.append(raw); sub_score.append(float(apart[1])); sub_tags.append(tags); sub_label_hr.append(tags_to_human_apart(tags))
        else:
            sub_raw.append(None); sub_score.append(None); sub_tags.append(None); sub_label_hr.append(None)

    if LABELS is not None:
        for i,(lbl, t) in enumerate(zip(final_lbl, texts)):
            if lbl == "Other":
                res = process.extractOne(t, LABELS, scorer=fuzz.token_set_ratio)
                if res and res[1] >= 80:
                    if sub_tags[i] is None: sub_tags[i] = []
                    sub_tags[i].append(f"hint: {res[0]}")

    out = pd.DataFrame({
        "text": texts,
        "pred_label": final_lbl,
        "pred_score": final_score,
        "sub_label": sub_label_hr,
        "sub_tags": sub_tags,
        "sub_label_raw": sub_raw,
        "sub_score": sub_score,
        "ood_dist2": dist2,
        "ood_z": z
    })
    return out


## 6. CSV-помощники

In [6]:

TEXT_CAND = ["text","description","title","content","message","body","full_text"]
LABEL_CAND = ["category","label","class","target","cat"]

def pick_text_col(df):
    for c in TEXT_CAND:
        if c in df.columns: return c
    return df.columns[0]

def pick_label_col(df):
    for c in LABEL_CAND:
        if c in df.columns: return c
    return None

def predict_csv(input_csv, text_col=None, out_path="predictions.csv"):
    df = pd.read_csv(input_csv)
    tcol = text_col or pick_text_col(df)
    preds = predict_texts(df[tcol].astype(str).tolist())
    out = pd.concat([df, preds], axis=1)
    out.to_csv(out_path, index=False)
    return out

from sklearn.metrics import classification_report, f1_score

def evaluate_csv(input_csv, text_col=None, label_col=None):
    df = pd.read_csv(input_csv)
    tcol = text_col or pick_text_col(df)
    lcol = label_col or pick_label_col(df)
    assert lcol is not None, "Нет столбца с истинной меткой"
    preds = predict_texts(df[tcol].astype(str).tolist())
    mask = preds["pred_label"] != "Other"
    y_true = df.loc[mask, lcol].astype(str).values
    y_pred = preds.loc[mask, "pred_label"].values
    print(classification_report(y_true, y_pred, digits=4))
    print("Macro F1 (без Other):", round(f1_score(y_true, y_pred, average="macro"), 4))
    return preds


## 7. Smoke-test

In [7]:

_ = predict_texts(["Продам iPhone 12, состояние идеал", "Сдам 2-комнатную квартиру в центре"], top_k=5).head(2)
_


,text,pred_label,pred_score,sub_label,sub_tags,sub_label_raw,sub_score,ood_dist2,ood_z
0,"Продам iPhone 12, состояние идеал",Смартфоны,0.529856,None,None,None,NaN,380.140717,-0.714740
1,Сдам 2-комнатную квартиру в центре,Квартиры — аренда,0.468692,"Квартира — аренда, 2-комнатная","[квартира: аренда, комнат: 2]",аренда_2,0.999015,382.413116,-0.711378


## 8. Ручные sanity-тесты

In [16]:

samples = [
    # Авто
    "Продаю автомобиль Toyota Camry 2019, автомат, один хозяин",
    "Продам Kia Rio, 1.6, пробег 52 тыс, без вложений",
    "BMW 3 серии, 2016 год, М-пакет, обмен возможен",
    "Лада Веста СВ Кросс, состояние отличное",
    "Audi A6 2015, полный привод, пробег 120 тыс",
    "Mercedes-Benz E200, 2020 год, AMG пакет",
    "Hyundai Solaris 2018, один хозяин",
    "Nissan X-Trail, 2019, полный привод, состояние нового",
    "Продам Lada Granta, 2021, без пробега",
    "Mazda CX-5, 2017, автомат, хорошее состояние",
    "Ford Focus 2, 2008, универсал, срочно",
    "Skoda Octavia 2016, DSG, возможен обмен",
    "Chevrolet Niva 2014, 4x4, торг уместен",
    "Opel Astra H, 2012, пробег 180 тыс",
    "Продаю запчасти для авто, оригинал OEM, подходят на Kia/Hyundai",
    "Шины зимние, радиус 17, состояние отличное",
    "ТОЙТА Камри 2018, отличное состояние",
    "Volkswagen Polo 2021, АКПП, комплектация Life",

    # Электроника
    "Продам ноутбук Lenovo i5 8GB 256GB SSD, состояние отличное",
    "Смартфон Samsung Galaxy S22, полный комплект",
    "Смартфон Zeno X5 Pro, новый, торг уместен",
    "iPhone 12, 128GB, б/у, батарея 90%",
    "Пылесос Dyson, беспроводной, в отличном состоянии",
    "Стиральная машина Bosch, загрузка 6 кг",
    "Куплю перфоратор б/у, срочно",

    # Недвижимость
    "Квартира в Москве, 2 комнаты, продажа от собственника",
    "Сдаю комнату в Санкт-Петербурге на длительный срок",
    "Сдам 2-комнатную квартиру в центре",
    "Сдам студию у метро, можно с животными",
    "Продажа 1-к квартиры, новостройка, ключи на руках",
    "Аренда 3-к квартиры на Невском, евроремонт",
    "Продаю однокомнатную квартиру в центре Москвы",
    "Сдам трехкомнатную квартиру на длительный срок",
    "Продам студию в новостройке у метро",
    "Аренда 2-комнатной квартиры, евроремонт, мебель",
    "Квартира в новостройке, продажа, 4 комнаты",
    "Сдаeтся 1-к квартира рядом с парком",
    "Продаeтся квартира-студия, ЖК Солнечный",
    "Сдам 2к квартиру с мебелью, без животных",
    "Продажа 3-комнатной квартиры в центре",
    "Куплю квартиру 2 комнаты, срочно",

    # Прочее/спам
    "кликайте по ссылке и выигрывайте айфон 15 бесплатно",
    "Заработок без вложений, пиши в телеграм",
]


preds = predict_texts(samples, top_k=5)
pd.set_option("display.max_colwidth", 120)

def _is_ood_row(row):
    cond_z = row["ood_z"] > z_thr
    cond_d = (thr_raw is not None) and (row["ood_dist2"] > thr_raw)
    return bool(cond_z or cond_d)

view = preds.copy()
view["is_other"] = view["pred_label"].eq("Other")
view["is_ood"] = view.apply(_is_ood_row, axis=1)

cols = ["text","pred_label","pred_score","sub_label","sub_tags","sub_label_raw","sub_score","is_ood","ood_z","ood_dist2"]
display(view[cols].head(25))


,text,pred_label,pred_score,sub_label,sub_tags,sub_label_raw,sub_score,is_ood,ood_z,ood_dist2
0,"Продаю автомобиль Toyota Camry 2019, автомат, один хозяин",Легковые автомобили,0.393584,Авто — Toyota,[марка: Toyota],Toyota,0.997827,False,-0.718652,377.496918
1,"Продам Kia Rio, 1.6, пробег 52 тыс, без вложений",Other,0.175918,None,None,None,NaN,False,-0.745670,359.237671
2,"BMW 3 серии, 2016 год, М-пакет, обмен возможен",Other,0.141974,None,None,None,NaN,False,-0.738975,363.762085
3,"Лада Веста СВ Кросс, состояние отличное",Other,0.298968,None,None,None,NaN,False,-0.714242,380.477173
4,"Audi A6 2015, полный привод, пробег 120 тыс",Other,0.166390,None,None,None,NaN,False,-0.730363,369.582245
5,"Mercedes-Benz E200, 2020 год, AMG пакет",Other,0.268501,None,None,None,NaN,False,-0.744924,359.741577
6,"Hyundai Solaris 2018, один хозяин",Other,0.131149,None,None,None,NaN,False,-0.743938,360.408386
7,"Nissan X-Trail, 2019, полный привод, состояние нового",Other,0.199955,None,None,None,NaN,False,-0.721113,375.833740
8,"Продам Lada Granta, 2021, без пробега",Other,0.190340,None,None,None,NaN,False,-0.715087,379.906738
9,"Mazda CX-5, 2017, автомат, хорошее состояние",Other,0.172674,None,None,None,NaN,False,-0.718851,377.362671


## 9. Разрезы

In [9]:

print("== Other =="); 
display(view[view["pred_label"].eq("Other")][["text","pred_score","sub_tags","is_ood"]].head(20))

print("\n== Авто ==");
display(view[view["pred_label"].str.contains("легков", case=False, na=False)][["text","pred_score","sub_label","sub_tags"]].head(20))

print("\n== Квартиры ==");
display(view[view["pred_label"].str.contains("квартир", case=False, na=False)][["text","pred_score","sub_label","sub_tags"]].head(20))


== Other ==


,text,pred_score,sub_tags,is_ood
1,"Продам Kia Rio, 1.6, пробег 52 тыс, без вложений",0.175918,None,False
2,"BMW 3 серии, 2016 год, М-пакет, обмен возможен",0.141974,None,False
3,"Лада Веста СВ Кросс, состояние отличное",0.298968,None,False
6,"ТОЙТА Камри 2018, отличное состояние",0.052203,None,False
7,"Volkswagen Polo 2021, АКПП, комплектация Life",0.202724,None,False
12,"Пылесос Dyson, беспроводной, в отличном состоянии",0.159120,None,False
14,"Куплю перфоратор б/у, срочно",0.127378,None,False
18,"Сдам студию у метро, можно с животными",0.181931,None,False
21,кликайте по ссылке и выигрывайте айфон 15 бесплатно,0.164130,None,False
22,"Заработок без вложений, пиши в телеграм",0.235866,None,False



== Авто ==


,text,pred_score,sub_label,sub_tags
0,"Продаю автомобиль Toyota Camry 2019, автомат, один хозяин",0.393584,Авто — Toyota,[марка: Toyota]



== Квартиры ==


,text,pred_score,sub_label,sub_tags
15,"Квартира в Москве, 2 комнаты, продажа от собственника",0.423174,"Квартира — продажа, 2-комнатная","[квартира: продажа, комнат: 2]"
17,Сдам 2-комнатную квартиру в центре,0.468692,"Квартира — аренда, 2-комнатная","[квартира: аренда, комнат: 2]"
19,"Продажа 1-к квартиры, новостройка, ключи на руках",0.476528,"Квартира — продажа, 1-комнатная","[квартира: продажа, комнат: 1]"
20,"Аренда 3-к квартиры на Невском, евроремонт",0.697251,"Квартира — продажа, 3-комнатная","[квартира: продажа, комнат: 3]"


## 10. Саб-классы: targeted-тесты на CSV

In [10]:

from pathlib import Path

def find_path(cands):
    for p in cands:
        if Path(p).exists(): return str(p)
    return None

autos_csv = find_path(["data/autos_subclf_30000.csv", "/mnt/data/autos_subclf_30000.csv"])
apart_csv = find_path(["data/apartments_subclf_30000.csv", "/mnt/data/apartments_subclf_30000.csv"])

def run_subcheck(csv_path, sample_n=80):
    df = pd.read_csv(csv_path)
    tcol = next((c for c in TEXT_CAND if c in df.columns), df.columns[0])
    ex = df.sample(min(sample_n, len(df)), random_state=42)[tcol].astype(str).tolist()
    res = predict_texts(ex, top_k=5)
    return res.loc[res["sub_label"].notna(), ["text","pred_label","sub_label","sub_tags","sub_score"]]

if autos_csv:
    print("AUTOS csv:", autos_csv)
    out_autos = run_subcheck(autos_csv, sample_n=80)
    display(out_autos.head(25))
    print("Итого (autos) с саб-меткой:", len(out_autos))

if apart_csv:
    print("\nAPARTMENTS csv:", apart_csv)
    out_apart = run_subcheck(apart_csv, sample_n=80)
    display(out_apart.head(25))
    print("Итого (apartments) с саб-меткой:", len(out_apart))

if not autos_csv and not apart_csv:
    print("CSV для саб-классов не найдены — пропуск.")


CSV для саб-классов не найдены — пропуск.


## 11. Батч-оценка

In [11]:

from glob import glob
cand_files = sorted(glob("data/*10000_v2.csv") + glob("data/*_v2.csv") + glob("/mnt/data/*10000_v2.csv"))
eval_path = cand_files[0] if cand_files else None
print("Датасет:", eval_path if eval_path else "не найден")

if eval_path:
    df = pd.read_csv(eval_path)
    tcol = next((c for c in TEXT_CAND if c in df.columns), df.columns[0])
    lcol = next((c for c in LABEL_CAND if c in df.columns), None)

    N_ROWS = min(2000, len(df))
    sample_df = df.sample(N_ROWS, random_state=7)

    preds = predict_texts(sample_df[tcol].astype(str).tolist(), top_k=5)
    out = pd.concat([sample_df.reset_index(drop=True), preds], axis=1)

    other_rate = (out["pred_label"] == "Other").mean()
    print("Всего:", len(out), "| Доля Other:", round(other_rate, 4))

    if lcol:
        from sklearn.metrics import f1_score, classification_report
        mask = out["pred_label"] != "Other"
        y_true = out.loc[mask, lcol].astype(str).values
        y_pred = out.loc[mask, "pred_label"].values
        print("Macro F1 (без Other):", round(f1_score(y_true, y_pred, average="macro"), 4))
        print("\nОтчeт:")
        print(classification_report(y_true, y_pred, digits=4))

    display(out[["text","pred_label","pred_score","sub_label","sub_tags","sub_score","ood_z","ood_dist2"]].head(25))
else:
    print("Пропуск — общий датасет не найден.")


Датасет: не найден
Пропуск — общий датасет не найден.


## 12. Стресс-тест с шумом

In [12]:

import random

base = [
    "toyota camry 2017 автомат пробег 80 тыс отличное состояние",
    "iphone 13 pro max 256gb без сколов полный комплект",
    "сдам 1к квартиру у метро на длительный срок",
    "куплю зимние шины r16",
    "ноутбук hp i7 16gb ram ssd 512",
]
def noiser(s):
    s = s.lower()
    if random.random() < 0.6:
        i = random.randrange(len(s))
        s = s[:i] + ("" if random.random()<0.5 else s[i]) + s[i:]
    if random.random() < 0.6:
        s = s + " !!! срочно торг"[:random.randint(0,15)]
    if random.random() < 0.3:
        s = re.sub(r"[aeiou]", "x", s)
    return s

stress = [noiser(random.choice(base)) for _ in range(40)]
stress_preds = predict_texts(stress, top_k=5)
display(stress_preds[["text","pred_label","pred_score","sub_label","sub_tags","sub_score","ood_z"]].head(25))
print("Доля Other на шуме:", round((stress_preds["pred_label"]=="Other").mean(), 4))


,text,pred_label,pred_score,sub_label,sub_tags,sub_score,ood_z
0,toyota camry 2017 автомат пробег 80 тыс отличное состояние !!! с,Other,0.090870,None,None,NaN,-0.757608
1,нооутбук hp x7 16gb rxm ssd 512 !!!,Ноутбуки,0.585552,None,None,NaN,-0.715404
2,ноутбук hp i7 16gb ram ssd 512,Ноутбуки,0.676756,None,None,NaN,-0.717461
3,ноутбук hp i7 16gb ram ssd 512,Ноутбуки,0.676756,None,None,NaN,-0.717461
4,xphhxnx 13 prx mxx 256gb без сколов полный комплект,Other,0.087467,None,None,NaN,-0.772610
5,ноутбук hp x7 16gb rxm ssd 512 !!!,Ноутбуки,0.677118,None,None,NaN,-0.716287
6,купллю зимние шины r16 !!! ср,Шины и диски,0.924856,None,None,NaN,-0.736939
7,ноутбук hp i7 16gb ram ssd 512,Ноутбуки,0.676756,None,None,NaN,-0.717461
8,сдам 1к квартиру у метро на длительный срок !!! срочно т,Квартиры — аренда,0.696674,"Квартира — аренда, 1-комнатная","[квартира: аренда, комнат: 1]",0.998767,-0.713021
9,куплю зимние шины r16,Шины и диски,0.922527,None,None,NaN,-0.736171


Доля Other на шуме: 0.325


## 13. CLI-mini

In [ ]:

def demo():
    print("Введите текст (Enter — выход):")
    while True:
        try:
            s = input("> ").strip()
        except EOFError:
            break
        if not s: break
        r = predict_texts([s], top_k=5).iloc[0]
        print(f"[coarse] {r['pred_label']} ({r['pred_score']:.3f}) | [sub] {r['sub_label']} | tags={r['sub_tags']} | OOD z={r['ood_z']:.2f}")
demo()


In [14]:

import numpy as np, pandas as pd
from itertools import chain

assert 'view' in globals(), "Сначала запусти раздел 8 (создается DataFrame `view`)."
df = view.copy()

def pct(x): return round(100*float(x), 1)

summary = {
    "N": len(df),
    "Other_rate_%": pct((df["pred_label"]=="Other").mean()),
    "OOD_rate_%": pct(df["is_ood"].mean()),
    "Mean_pred_score": round(df["pred_score"].mean(), 3),
    "Sub_coverage_% (any)": pct(df["sub_label"].notna().mean()),
    "Sub_autos_%": pct(df.loc[df["pred_label"].str.contains("легков", case=False, na=False), "sub_label"].notna().mean()
                       if (df["pred_label"].str.contains("легков", case=False, na=False)).any() else 0),
    "Sub_aparts_%": pct(df.loc[df["pred_label"].str.contains("квартир", case=False, na=False), "sub_label"].notna().mean()
                        if (df["pred_label"].str.contains("квартир", case=False, na=False)).any() else 0),
}
display(pd.Series(summary, name="Summary"))

# Проверка: комнаты не триггерят саб-квартиры
rooms_check = df[df["text"].str.contains("комнат", case=False, na=False)]
if len(rooms_check):
    print("\n[Check] Примеры с 'комнат': саб-метка должна быть только если coarse=Квартиры — ...")
    display(rooms_check[["text","pred_label","sub_label","sub_tags"]].head(10))

# OOD распределение
print("\nOOD z-score describe:")
display(df["ood_z"].describe(percentiles=[.5,.9,.95,.99]))
print("\nOOD ||CLS||^2 describe:")
display(df["ood_dist2"].describe(percentiles=[.5,.9,.95,.99]))

# Топ тегов сабов
tags = list(chain.from_iterable([t for t in df["sub_tags"] if isinstance(t, list)]))
if tags:
    print("\nTop sub-tags:")
    display(pd.Series(tags).value_counts().head(15))


N                        23.000
Other_rate_%             43.500
OOD_rate_%                0.000
Mean_pred_score           0.435
Sub_coverage_% (any)     21.700
Sub_autos_%             100.000
Sub_aparts_%            100.000
Name: Summary, dtype: float64


[Check] Примеры с 'комнат': саб-метка должна быть только если coarse=Квартиры — ...


,text,pred_label,sub_label,sub_tags
15,"Квартира в Москве, 2 комнаты, продажа от собственника",Квартиры — продажа,"Квартира — продажа, 2-комнатная","[квартира: продажа, комнат: 2]"
16,Сдаю комнату в Санкт-Петербурге на длительный срок,Комнаты — аренда,None,None
17,Сдам 2-комнатную квартиру в центре,Квартиры — аренда,"Квартира — аренда, 2-комнатная","[квартира: аренда, комнат: 2]"



OOD z-score describe:


count    23.000000
mean     -0.723703
std       0.015058
min      -0.752219
50%      -0.718603
90%      -0.709830
95%      -0.706505
99%      -0.705515
max      -0.705335
Name: ood_z, dtype: float64


OOD ||CLS||^2 describe:


count     23.000000
mean     374.083527
std       10.176423
min      354.811493
50%      377.530090
90%      383.459271
95%      385.706378
99%      386.375196
max      386.496979
Name: ood_dist2, dtype: float64


Top sub-tags:


квартира: продажа    3
комнат: 2            2
марка: Toyota        1
квартира: аренда     1
комнат: 1            1
комнат: 3            1
Name: count, dtype: int64

## 14. A/B + Sub Re-run

In [26]:

#  A/B + Sub Re-run: 
#    - мягче tau_other для авто/смартфонов + лексикон(+fuzzy),

import re, numpy as np, pandas as pd
from IPython.display import display
from rapidfuzz import fuzz

# Настройки гипотезы

# базовый порог для авто при переходе из Other (A/B)
TAU_AUTO = 0.12
# "loose"-порог для сильных авто-хинтов (Camry/BMW/Solaris/Astra)
TAU_AUTO_LOOSE = 0.08
# порог для смартфонов при переходе из Other
TAU_PHONE = 0.20
# минимальная уверенность, если есть уверенный лексикон-хинт
BOOST_SCORE = 0.55   
# порог fuzzy-сходства
FUZZY_THR = 86

# Саб-пороги
TAU_AUTOS = float(globals().get("TAU_AUTOS", 0.83))
TAU_APART = float(globals().get("TAU_APART", 0.45))

BRAND_HINTS = {
    "Смартфоны": [
        "iphone","iphon","айфон","samsung","galaxy","xiaomi","redmi",
        "huawei","honor","pixel","oneplus","oppo","realme"
    ],
    "Легковые автомобили": [
        "toyota","тойота","camry","камри","kia","киа","rio","bmw","бмв","mercedes","мерседес","audi","ауди",
        "vw","volkswagen","фольксваген","поло","lada","лада","vesta","веста","mazda","ниссан","nissan",
        "hyundai","хендай","хeндэ","chevrolet","шевроле","skoda","шкода","opel","опель","ford","renault","рено",
        "honda","mitsubishi","митсубиси","mitsubisi","тойта"  # частые опечатки
    ]
}
STRONG_AUTO = ["камри","camry","bmw","солярис","solaris","astra","астра"]

def _has_hint(txt: str, keys: list, fuzzy_thr: int = FUZZY_THR) -> bool:
    t = txt.lower()
    if any(k in t for k in keys):
        return True
    if fuzz is not None:
        for k in keys:
            if fuzz.partial_ratio(t, k) >= fuzzy_thr:
                return True
    return False

def _get_series(df: pd.DataFrame, col: str) -> pd.Series:
    if col not in df.columns:
        raise KeyError(f"Column '{col}' not found.")
    mask = (df.columns == col)
    if mask.sum() == 1:
        return df[col]
    idx = np.where(mask)[0][-1] # последняя колонка с таким именем
    return df.iloc[:, idx]

def _summarize(df: pd.DataFrame, label_col="pred_label") -> pd.Series:
    s_lbl = _get_series(df, label_col).astype(str)
    s_score_col = label_col.replace("label", "score")
    s_score = _get_series(df, s_score_col) if s_score_col in df.columns else pd.Series([np.nan]*len(df))
    return pd.Series({
        "N": len(df),
        "Other_%": round(100 * (s_lbl.eq("Other").mean()), 1),
        "Autos_%": round(100 * (s_lbl.str.contains("легков", case=False, na=False).mean()), 1),
        "Phones_%": round(100 * (s_lbl.str.contains("смартф", case=False, na=False).mean()), 1),
        "MeanScore": round(float(np.nanmean(pd.to_numeric(s_score, errors="coerce"))), 3),
    }, name=label_col)

def _apart_humanize(raw: str):
    if raw is None or not isinstance(raw, str):
        return None, None
    t = raw.strip().lower()
    if t.startswith("студия"):
        parts = t.split("_")
        mode = parts[1] if len(parts) > 1 else None
        label = f"Квартира — {('аренда' if mode=='аренда' else 'продажа' if mode=='продажа' else 'студия')}, студия"
        tags = [f"квартира: {mode or 'студия'}", "комнат: 0"]
        return label, tags
    if "_" in t:
        mode, rooms = t.split("_", 1)
        try: r = int(rooms)
        except: r = rooms
        label = f"Квартира — {mode}, {r}-комнатная"
        tags = [f"квартира: {mode}", f"комнат: {r}"]
        return label, tags
    return f"Квартира — {t}", [f"квартира: {t}"]

def _auto_humanize(brand: str):
    if not brand: return None, None
    return f"Авто — {brand}", [f"марка: {brand}"]

def apply_hypothesis_on_df(df_base: pd.DataFrame,
                           tau_auto=TAU_AUTO, tau_auto_loose=TAU_AUTO_LOOSE,
                           tau_phone=TAU_PHONE, boost_score=BOOST_SCORE,
                           strong_auto_keys=STRONG_AUTO) -> pd.DataFrame:
    df = df_base.copy()
    new_labels, new_scores = [], []
    auto_hint, phone_hint = [], []

    for _, row in df.iterrows():
        lbl = str(row["pred_label"])
        score= float(row["pred_score"])
        txt = str(row["text"]).strip()
        t = txt.lower()

        has_auto = _has_hint(t, BRAND_HINTS["Легковые автомобили"])
        has_phone = _has_hint(t, BRAND_HINTS["Смартфоны"])
        auto_hint.append(has_auto); phone_hint.append(has_phone)

        # сильный авто-хинт?
        strong_auto = has_auto and any(k in t for k in strong_auto_keys)
        tau_auto_eff = tau_auto if not strong_auto else min(tau_auto, tau_auto_loose)

        # Перекласс из Other -> авто/смартфоны
        if lbl == "Other":
            if has_auto and score >= tau_auto_eff:
                lbl = "Легковые автомобили"
            elif has_phone and score >= tau_phone:
                lbl = "Смартфоны"

        # Лексикон-буст уверенности
        if (lbl == "Легковые автомобили" and has_auto) or (lbl == "Смартфоны" and has_phone):
            score = max(score, boost_score)

        new_labels.append(lbl)
        new_scores.append(score)

    df["has_auto_hint"]   = auto_hint
    df["has_phone_hint"]  = phone_hint
    df["pred_label_hypo"] = new_labels
    df["pred_score_hypo"] = new_scores
    return df

# Baseline
try:
    samples
except NameError:
    samples = [
        # авто
        "Продаю автомобиль Toyota Camry 2019, автомат, один хозяин",
        "Продам Kia Rio, 1.6, пробег 52 тыс, без вложений",
        "BMW 3 серии, 2016 год, М-пакет, обмен возможен",
        "Лада Веста СВ Кросс, состояние отличное",
        "Audi A6 2015, полный привод, пробег 120 тыс",
        "Mercedes-Benz E200, 2020 год, AMG пакет",
        "Volkswagen Polo 2021, АКПП, комплектация Life",
        # телефоны
        "iPhone 12, 128GB, б/у, батарея 90%",
        "Смартфон Samsung Galaxy S22, полный комплект",
        # недвижимость
        "Квартира в Москве, 2 комнаты, продажа от собственника",
        "Сдам 2-комнатную квартиру в центре",
        "Сдам студию у метро, можно с животными",
        # прочее
        "кликайте по ссылке и выигрывайте айфон 15 бесплатно",
    ]

# baseline 
df_base = predict_texts(samples, top_k=5).copy()

# Гипотиза
df_hypo = apply_hypothesis_on_df(df_base)

# убираем дубли имeн колонок
df_after = (
    df_hypo
      .drop(columns=[c for c in ["pred_label","pred_score"] if c in df_hypo.columns], errors="ignore")
      .rename(columns={"pred_label_hypo":"pred_label", "pred_score_hypo":"pred_score"})
      .copy()
)

# Сводка до/после
print("=== Hypothesis settings ===")
print(f"tau_auto={TAU_AUTO} | tau_auto_loose={TAU_AUTO_LOOSE} | tau_phone={TAU_PHONE} | "
      f"boost_score={BOOST_SCORE} | fuzzy_thr={FUZZY_THR} | fuzzy_on={fuzz is not None} | "
      f"TAU_AUTOS={TAU_AUTOS} | TAU_APART={TAU_APART}")

print("\n=== Summary (baseline vs hypothesis) ===")
base_sum = _summarize(df_base, "pred_label")
hypo_sum = _summarize(df_after, "pred_label")
display(pd.concat([base_sum, hypo_sum], axis=1))

# Распределение классов до/после
print("\n=== Value counts (top-15) ===")
vc_before = _get_series(df_base, "pred_label").value_counts().head(15)
vc_after = _get_series(df_after, "pred_label").value_counts().head(15)
display(pd.DataFrame({"before": vc_before, "after": vc_after}).fillna(0).astype(int))

# Изменившиеся предсказания
changed = df_hypo[df_hypo["pred_label_hypo"] != df_hypo["pred_label"]].copy()
print(f"\nChanged predictions: {len(changed)}")
if len(changed):
    display(changed[["text","pred_label","pred_score","pred_label_hypo","pred_score_hypo","sub_label"]].head(100))

# Дозапуск саб-классификаторов для строк, переведeнных гипотезой

# гарантируем колонки и их типы
for col in ["sub_label","sub_score","sub_tags"]:
    if col not in df_after.columns:
        df_after[col] = np.nan
df_after["sub_tags"] = df_after["sub_tags"].astype(object)

allowed_coarse = {"Легковые автомобили", "Квартиры — аренда", "Квартиры — продажа"}

need_mask = df_after["pred_label"].isin(allowed_coarse) & (df_after["sub_label"].isna() | df_after["sub_label"].eq(None))
texts_need = df_after.loc[need_mask, "text"].tolist()
coarse_need = df_after.loc[need_mask, "pred_label"].tolist()

recomputed = 0
try:
    if texts_need:
        autos_sub, apart_sub = sub_infer(texts_need, coarse_need)  # из основного пайплайна

        new_sub_label = []; new_sub_score = []; new_sub_tags = []
        for coarse, a, ap in zip(coarse_need, autos_sub, apart_sub):
            human, tags, score = None, None, None
            if coarse == "Легковые автомобили" and a is not None and float(a[1]) >= TAU_AUTOS:
                human, tags = _auto_humanize(a[0]); score = float(a[1])
            elif coarse.startswith("Квартиры") and ap is not None and float(ap[1]) >= TAU_APART:
                human, tags = _apart_humanize(ap[0]); score = float(ap[1])
            new_sub_label.append(human); new_sub_score.append(score); new_sub_tags.append(tags)

        # аккуратно мeржим tags и записываем сериями (dtype=object)
        base_tags = df_after.loc[need_mask, "sub_tags"]
        merged = []
        for old, add in zip(base_tags.tolist(), new_sub_tags):
            if isinstance(old, list) and add:
                merged.append(list(dict.fromkeys(old + add)))
            else:
                merged.append(add)

        idx_need = df_after.index[need_mask]
        df_after.loc[idx_need, "sub_label"] = pd.Series(new_sub_label, index=idx_need, dtype=object)
        df_after.loc[idx_need, "sub_score"] = pd.Series(new_sub_score, index=idx_need, dtype=float)
        df_after.loc[idx_need, "sub_tags"]  = pd.Series(merged,        index=idx_need, dtype=object)
        recomputed = len(idx_need)
except NameError:
    print("\n[warn] sub_infer() не найден — пропускаю пересчeт саб-классов.")

print(f"\nRecomputed sub-classes for {recomputed} rows.")

# Витрина: только то, что стало авто/квартиры после гипотезы и получило саб 
promoted_mask = (df_base["pred_label"] != df_after["pred_label"]) & df_after["pred_label"].isin(allowed_coarse)
view_enriched = df_after.loc[promoted_mask, ["text","pred_label","pred_score","sub_label","sub_score","sub_tags"]]
print("\n=== Promoted & enriched rows ===")
display(view_enriched.head(30))

# Превью (до/после) 
print("\n=== Preview BEFORE (first 30) ===")
display(df_base[["text","pred_label","pred_score","sub_label"]].head(30))
print("\n=== Preview AFTER  (first 30) ===")
display(df_after[["text","pred_label","pred_score","sub_label"]].head(30))


=== Hypothesis settings ===
tau_auto=0.12 | tau_auto_loose=0.08 | tau_phone=0.2 | boost_score=0.55 | fuzzy_thr=86 | fuzzy_on=True | TAU_AUTOS=0.8341092920652331 | TAU_APART=0.44362904422064614

=== Summary (baseline vs hypothesis) ===


,pred_label,pred_label
N,43.000,43.00
Other_%,55.800,23.30
Autos_%,2.300,34.90
Phones_%,7.000,7.00
MeanScore,0.367,0.49



=== Value counts (top-15) ===


,before,after
pred_label,,
Other,24,10
Запчасти для авто,1,1
Квартиры — аренда,6,6
Квартиры — продажа,4,4
Комнаты — аренда,1,1
Легковые автомобили,1,15
Ноутбуки,1,1
Смартфоны,3,3
Стиральные машины,1,1



Changed predictions: 14


...

,text,pred_label,pred_score,sub_label,sub_score,sub_tags
1,"Продам Kia Rio, 1.6, пробег 52 тыс, без вложений",Легковые автомобили,0.55,Авто — Kia,0.997923,[марка: Kia]
2,"BMW 3 серии, 2016 год, М-пакет, обмен возможен",Легковые автомобили,0.55,Авто — BMW,0.997680,[марка: BMW]
3,"Лада Веста СВ Кросс, состояние отличное",Легковые автомобили,0.55,None,NaN,None
4,"Audi A6 2015, полный привод, пробег 120 тыс",Легковые автомобили,0.55,Авто — Audi,0.997417,[марка: Audi]
5,"Mercedes-Benz E200, 2020 год, AMG пакет",Легковые автомобили,0.55,Авто — Mercedes,0.997408,[марка: Mercedes]
6,"Hyundai Solaris 2018, один хозяин",Легковые автомобили,0.55,Авто — Hyundai,0.997333,[марка: Hyundai]
7,"Nissan X-Trail, 2019, полный привод, состояние нового",Легковые автомобили,0.55,Авто — Nissan,0.997672,[марка: Nissan]
8,"Продам Lada Granta, 2021, без пробега",Легковые автомобили,0.55,Авто — Lada,0.997794,[марка: Lada]
9,"Mazda CX-5, 2017, автомат, хорошее состояние",Легковые автомобили,0.55,Авто — Mazda,0.997538,[марка: Mazda]
10,"Ford Focus 2, 2008, универсал, срочно",Легковые автомобили,0.55,Авто — Ford,0.997644,[марка: Ford]



=== Preview BEFORE (first 30) ===


,text,pred_label,pred_score,sub_label
0,"Продаю автомобиль Toyota Camry 2019, автомат, один хозяин",Легковые автомобили,0.393584,Авто — Toyota
1,"Продам Kia Rio, 1.6, пробег 52 тыс, без вложений",Other,0.175918,None
2,"BMW 3 серии, 2016 год, М-пакет, обмен возможен",Other,0.141974,None
3,"Лада Веста СВ Кросс, состояние отличное",Other,0.298968,None
4,"Audi A6 2015, полный привод, пробег 120 тыс",Other,0.166390,None
5,"Mercedes-Benz E200, 2020 год, AMG пакет",Other,0.268501,None
6,"Hyundai Solaris 2018, один хозяин",Other,0.131149,None
7,"Nissan X-Trail, 2019, полный привод, состояние нового",Other,0.199955,None
8,"Продам Lada Granta, 2021, без пробега",Other,0.190340,None
9,"Mazda CX-5, 2017, автомат, хорошее состояние",Other,0.172674,None



=== Preview AFTER  (first 30) ===


,text,pred_label,pred_score,sub_label
0,"Продаю автомобиль Toyota Camry 2019, автомат, один хозяин",Легковые автомобили,0.550000,Авто — Toyota
1,"Продам Kia Rio, 1.6, пробег 52 тыс, без вложений",Легковые автомобили,0.550000,Авто — Kia
2,"BMW 3 серии, 2016 год, М-пакет, обмен возможен",Легковые автомобили,0.550000,Авто — BMW
3,"Лада Веста СВ Кросс, состояние отличное",Легковые автомобили,0.550000,None
4,"Audi A6 2015, полный привод, пробег 120 тыс",Легковые автомобили,0.550000,Авто — Audi
5,"Mercedes-Benz E200, 2020 год, AMG пакет",Легковые автомобили,0.550000,Авто — Mercedes
6,"Hyundai Solaris 2018, один хозяин",Легковые автомобили,0.550000,Авто — Hyundai
7,"Nissan X-Trail, 2019, полный привод, состояние нового",Легковые автомобили,0.550000,Авто — Nissan
8,"Продам Lada Granta, 2021, без пробега",Легковые автомобили,0.550000,Авто — Lada
9,"Mazda CX-5, 2017, автомат, хорошее состояние",Легковые автомобили,0.550000,Авто — Mazda


## Итоги (A2 · Archived Ensemble)

- Качество на sanity/стрессе приличное, но **латентность и стоимость выше целевых** из-за нескольких тяжёлых BERT-голов.
    
- **Ключевой урок:** тонкие головы на фиксированных эмбеддингах дают сравнимое качество на саб-задачах при **значительно меньшей цене**.
    

**Решение**

- Держим ноутбук как Appendix и контрольную точку для регрессионных тестов.
    
- Боевой пайплайн — см. `04_SubHeads_Autos_Aparts_ThinHeads.ipynb` и `05_Inference_Pipeline_Final.ipynb`.
    

**Дальше**

- Если вернём CE, завести флаг и early-exit по margin; пороги/калибровку — в отдельном json.